In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd

In [3]:
data_path = '/kaggle/input/commonlit-evaluate-student-summaries/'

In [4]:
# prompts train
train_pro = pd.read_csv(data_path + 'prompts_train.csv')
train_pro

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


In [5]:
# summaries train
train_sum = pd.read_csv(data_path + 'summaries_train.csv')
train_sum

,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757
...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,They used all sorts of chemical concoctions to...,0.205683,0.380538
7161,ffc34d056498,3b9047,The lowest classes are slaves and farmers slav...,-0.308448,0.048171
7162,ffd1576d2e1b,3b9047,they sorta made people start workin...,-1.408180,-0.493603
7163,ffe4a98093b2,39c16e,An ideal tragety has three elements that make ...,-0.393310,0.627128


In [6]:
# we will need to take a portion of the train data as test data

train = train_pro.merge(train_sum , on = "prompt_id")
train

,prompt_id,prompt_question,prompt_title,prompt_text,student_id,text,content,wording
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00791789cc1f,1 element of an ideal tragedy is that it shoul...,-0.210614,-0.471415
1,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,0086ef22de8f,The three elements of an ideal tragedy are: H...,-0.970237,-0.417058
2,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,0094589c7a22,Aristotle states that an ideal tragedy should ...,-0.387791,-0.584181
3,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00cd5736026a,One element of an Ideal tragedy is having a co...,0.088882,-0.594710
4,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00d98b8ff756,The 3 ideal of tragedy is how complex you need...,-0.687288,-0.460886
...,...,...,...,...,...,...,...,...
7160,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",ff37545b2805,"In paragraph two, they would use pickle meat a...",1.520355,-0.292990
7161,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",ff4ed38ef099,"in the first paragraph it says ""either can it...",-1.204574,-1.169784
7162,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",ff53b94f7ce0,They would have piles of filthy meat on the fl...,0.328739,-1.053294
7163,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",ff7c7e70df07,They used all sorts of chemical concoctions to...,0.205683,0.380538


In [7]:
import warnings
import logging
import os

In [8]:
# disabling unnecceseray warnings
warnings.simplefilter("ignore")
logging.disable(logging.ERROR)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [9]:
import tensorflow as tf
from transformers import RobertaTokenizer, TFRobertaModel
from tensorflow import keras
from keras import layers
import datetime

In [10]:
# # Limit the GPU memory growth using TensorFlow
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [11]:
import random

# Set random seeds
random_seed = 42
np.random.seed(random_seed)
tf.random.set_seed(random_seed)
random.seed(random_seed)

In [12]:
!pip install autocorrect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622364 sha256=92eed6e66e72dd1043ddcb997aabc6ab8662d26aba7eecdd0cb0ddcd6c57ce85
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect


In [13]:
from autocorrect import Speller
spell = Speller(lang='en', fast=True)
spell('helo')

'held'

In [14]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [15]:
# Reload the TensorBoard notebook extension if needed
%reload_ext tensorboard

In [16]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta = TFRobertaModel.from_pretrained('roberta-base')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [17]:
# Auto correcting spellings of all summaries
sep = f" {tokenizer.sep_token} "
train["text"] = train["text"].apply(lambda x: spell(x))
train['input'] = train['prompt_title'] + sep + train['prompt_question'] + sep + train['text']
train['input']

0       On Tragedy </s> Summarize at least 3 elements ...
1       On Tragedy </s> Summarize at least 3 elements ...
2       On Tragedy </s> Summarize at least 3 elements ...
3       On Tragedy </s> Summarize at least 3 elements ...
4       On Tragedy </s> Summarize at least 3 elements ...
                              ...                        
7160    Excerpt from The Jungle </s> Summarize the var...
7161    Excerpt from The Jungle </s> Summarize the var...
7162    Excerpt from The Jungle </s> Summarize the var...
7163    Excerpt from The Jungle </s> Summarize the var...
7164    Excerpt from The Jungle </s> Summarize the var...
Name: input, Length: 7165, dtype: object

In [18]:
# Preprocessing features and labels

# Tokenize text data
MAX_SUMMARY_LENGTH = int(train['input'].apply(len).mean())
X_train = tokenizer.batch_encode_plus(train['input'].tolist(),
                                              add_special_tokens=True,
                                              truncation=True,
                                              padding='max_length',
                                              return_tensors='tf',
                                              max_length=MAX_SUMMARY_LENGTH,
                                              return_attention_mask = True)

Y_train = tf.constant(train[['content', 'wording']].values, dtype=tf.float32)

In [19]:
# print("X_train element:")
# print(X_train['input_ids'][:1])
# print(X_train['attention_mask'][:1])
# print()
# print("Y_train element:")
# print(Y_train[0])
# print()
# print(f"X_train shape: (2, {X_train['input_ids'].shape}, {X_train['attention_mask'].shape}) ")
# print(f"Y_train shape: {Y_train.shape}")

In [20]:
# Create a layer that wraps the RoBERTa model to support Keras library
class RoBERTaWrapper(layers.Layer):
    def __init__(self, roberta_model, **kwargs):
        super().__init__(**kwargs)
        self.roberta = roberta_model

    def call(self, inputs):
        roberta_input1 = inputs[0]
        roberta_input2 = inputs[1]

        # Call the roberta model and get the last hidden state
        roberta_output = self.roberta(input_ids=roberta_input1,
                                      attention_mask=roberta_input2)
        return roberta_output['last_hidden_state']

In [21]:
# Define keras neural network with Functional API
    
# Explanation of the Functional API: 
# For every layer we specify between parentheses at the end of the line the previous layer which will be the input.

input_layer1 = keras.Input(shape=(MAX_SUMMARY_LENGTH,), dtype='int32')
input_layer2 = keras.Input(shape=(MAX_SUMMARY_LENGTH,), dtype='int32')

roberta_output = RoBERTaWrapper(roberta)([input_layer1, input_layer2])
global_avg_pooling = layers.GlobalAveragePooling1D()(roberta_output)

hidden_layer1 = layers.Dense(256, activation='linear')(global_avg_pooling)
batch_norm1 = layers.BatchNormalization()(hidden_layer1)
hidden_layer2 = layers.Dense(128, activation='linear')(batch_norm1)
batch_norm2 = layers.BatchNormalization()(hidden_layer2)

output_layer = layers.Dense(2, activation='linear')(batch_norm2)

# Define the model
model = keras.Model(inputs=[input_layer1, input_layer2], outputs=output_layer)

# Make roberta params untrainable
for layer in model.layers[:-5]:
    layer.trainable = False
    
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 567)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 567)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ro_ber_ta_wrapper   │ (None, 567, 768)  │          0 │ input_layer[0][0… │
│ (RoBERTaWrapper)    │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 768)       │          0 │ ro_ber_ta_wrappe… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    196,864 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256)       │      1,024 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 2)         │        258 │ batch_normalizat… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 231,554 (904.51 KB)

 Trainable params: 230,786 (901.51 KB)

 Non-trainable params: 768 (3.00 KB)

In [22]:
# Train the model

In [23]:
# Compile the model (add loss, optimizer, metrics)

# The loss function
def mcrmse(y_true, y_pred):
    columnwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=0)
    return tf.reduce_mean(tf.sqrt(columnwise_mse), axis=-1)

model.compile(
    loss=mcrmse,
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['r2_score']
)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [24]:
# Clear any logs from previous runs
# !rm -rf ./logs/
# num_of_samples = 10

inputs = X_train['input_ids']
masks = X_train['attention_mask']
# Train the model

# model.load_weights('model/saved_model_weights.weights.h5')

model.fit(x=[inputs, masks],
          y=Y_train,
          epochs=10,
          batch_size=16,
          validation_split=0.2,
          callbacks=[tensorboard_callback],
          verbose=2)

# Create the 'model' directory if it does not exist
if not os.path.exists('model'):
    os.makedirs('model')
    
model.save_weights('model/saved_model_weights.weights.h5')

Epoch 1/10


I0000 00:00:1709760094.167348      89 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1709760094.209074      89 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1709760302.112324      90 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1709760310.280059      91 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1709760366.644460      90 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


359/359 - 297s - 827ms/step - loss: 0.6986 - r2_score: 0.5213 - val_loss: 0.6812 - val_r2_score: 0.4601
Epoch 2/10
359/359 - 257s - 716ms/step - loss: 0.6134 - r2_score: 0.6391 - val_loss: 0.5684 - val_r2_score: 0.6166
Epoch 3/10
359/359 - 258s - 718ms/step - loss: 0.5955 - r2_score: 0.6590 - val_loss: 0.5820 - val_r2_score: 0.5937
Epoch 4/10
359/359 - 258s - 718ms/step - loss: 0.5836 - r2_score: 0.6719 - val_loss: 0.5912 - val_r2_score: 0.5765
Epoch 5/10
359/359 - 258s - 718ms/step - loss: 0.5744 - r2_score: 0.6819 - val_loss: 0.5988 - val_r2_score: 0.5641
Epoch 6/10
359/359 - 257s - 717ms/step - loss: 0.5665 - r2_score: 0.6903 - val_loss: 0.5940 - val_r2_score: 0.5740
Epoch 7/10
359/359 - 258s - 718ms/step - loss: 0.5595 - r2_score: 0.6976 - val_loss: 0.6093 - val_r2_score: 0.5463
Epoch 8/10
359/359 - 258s - 717ms/step - loss: 0.5532 - r2_score: 0.7041 - val_loss: 0.6094 - val_r2_score: 0.5468
Epoch 9/10
359/359 - 258s - 718ms/step - loss: 0.5468 - r2_score: 0.7106 - val_loss: 0.6334

In [25]:
# !rm -rf ./model/saved_model_weights.weights.h5

In [26]:
# import shutil
# # Zip the logs directory for downloading it locally 
# shutil.make_archive("/kaggle/working/logs", "zip", "./logs")

In [27]:
# http://localhost:6006/
%tensorboard --logdir logs/fit --port=6006

In [28]:
# Demo of the model

In [29]:
# model = model.load_model('model/saved_model_weights.weights.h5')
def eval_summary(prompt, question, summary):
    spell(summary)
    sample = prompt + sep + question + sep + summary
    tokenized_summary = tokenizer(sample,
                                  add_special_tokens=True,
                                  truncation=True,
                                  padding='max_length',
                                  return_tensors='tf',
                                  max_length=MAX_SUMMARY_LENGTH,
                                  return_attention_mask = True)

    output = model([tokenized_summary['input_ids'], tokenized_summary['attention_mask']])
    print(f"Content: { output[0][0]} \nWording: {output[0][1]}")

In [30]:
sample_index = 8
prompt = train['prompt_title'][sample_index]
question = train['prompt_question'][sample_index]
summary = train['text'][sample_index]

print('Predicted Scores:')
eval_summary(prompt, question, summary)

print(f"\nReal Scores:\nContent: {train['content'][sample_index]}\nWording: {train['wording'][sample_index]}")

Predicted Scores:
Content: 0.5121203660964966 
Wording: 1.3364146947860718

Real Scores:
Content: 0.670351243803345
Wording: 1.55163281950375
